In [1]:
import pandas as pd
import numpy as np

In [17]:
# for processing
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# for bag-of-words
from sklearn import feature_extraction, feature_selection, model_selection, naive_bayes, pipeline, manifold, preprocessing, metrics
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

## for train test split
import imblearn

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JYM\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\JYM\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\JYM\AppData\Roaming\nltk_data...


In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
cfpb_df = pd.read_csv('../data/CFPB with Duplicate Marked.csv')
print(cfpb_df.columns)
print(cfpb_df.shape)

C:\Users\JYM\AppData\Local\Temp/ipykernel_20072/2119433820.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  cfpb_df = pd.read_csv('../data/CFPB with Duplicate Marked.csv')


Index(['Unnamed: 0', 'Date received', 'Product', 'Sub-product', 'Issue',
       'Sub-issue', 'Consumer complaint narrative', 'Company public response',
       'Company', 'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID', 'narr_len',
       'days_to_today', 'dupi_id', 'dupi_len'],
      dtype='object')
(1300361, 23)


In [5]:
%%time
# Drop duplicates based on 'dupi_id' column
cfpb_df = cfpb_df.drop_duplicates(subset='dupi_id')
print(cfpb_df.shape)

(1106587, 23)
Wall time: 445 ms


In [27]:
def unigram_preprocess_text(text):
    '''
    Preprocess text, remove stop words perform stemming and lematization.
    :parameter
        :param text: string - name of column containing text
        :param flg_stemm: bool - whether stemming is to be applied
        :param flg_lemm: bool - whether lemmitisation is to be applied
    :return
        cleaned text in str format
    '''
    
    cleaned_text = ''
    # try:
    ## get stop words
    ## our choices of stop words
    lst_stopwords = nltk.corpus.stopwords.words("english")
    ## lst_stopwords=None

    ## First we will clean up the XXXX maskings in the text created by CFPB
    text = re.sub(r'X+X', '', str(text).strip())

    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())

    ## Tokenize (convert from string to list)
    lst_text = text.split()

    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]

    ## Stemming (remove -ing, -ly, ...)
    ## This stemming parts works pretty bad, but is something good to have. Better than nothing.
    ps = nltk.stem.porter.PorterStemmer()
    lst_text = [ps.stem(word) for word in lst_text]

    ## Lemmatisation (convert the word into root word)
    ## Also, not very helpful, but functions better than stemming part.
    lem = nltk.stem.wordnet.WordNetLemmatizer()
    lst_text = [lem.lemmatize(word) for word in lst_text]

    ## back to string from list
    cleaned_text = " ".join(lst_text)
    # except:
    #     pass
    return cleaned_text

def get_unigram_narr(df):
    '''
    This is the group by helper function
    '''
    df['unigram_narr'] = df['Consumer complaint narrative'].apply(unigram_preprocess_text)
    return df
    

In [28]:
%%time
small_cfpb_df = cfpb_df[['Product', 'Issue', 'State', 'ZIP code','Complaint ID','Consumer complaint narrative']].sample(500).copy()
small_cfpb_df[['Product', 'Issue', 'State', 'ZIP code']] = small_cfpb_df[['Product', 'Issue', 'State', 'ZIP code']].fillna('')
small_cfpb_df = small_cfpb_df.groupby(['Product', 'Issue', 'State', 'ZIP code']).apply(func=get_unigram_narr)

Wall time: 1.13 s


In [29]:
small_cfpb_df

,Product,Issue,State,ZIP code,Complaint ID,Consumer complaint narrative,unigram_narr
435780,Student loan,Dealing with your lender or servicer,MA,2090.0,3453067,I started making monthly student loan payments...,start make monthli student loan payment loan c...
11760,"Credit reporting, credit repair services, or o...",Incorrect information on your report,VA,22191.0,4789107,I was deactivated from the XXXX platform becau...,deactiv platform report checkr checkr never se...
704463,Credit card or prepaid card,"Other features, terms, or problems",FL,34241,4706992,5/3 stole {$1000.00} from me and has been lyin...,53 stole 100000 lie year see attach pictur onl...
2934,Credit card or prepaid card,Problem when making payments,IL,60431.0,6396747,I have a comenity Ulta card XXXX. \nXXXX In XX...,comen ulta card 2022 made payment total 70000 ...
51189,"Money transfer, virtual currency, or money ser...",Money was not available when promised,FL,33056.0,3120184,I made a XXXX transfer for {$2300.00} from my ...,made transfer 230000 bank america check accoun...
...,...,...,...,...,...,...,...
203315,"Credit reporting, credit repair services, or o...",Problem with a credit reporting company's inve...,CA,95354.0,6637378,In accordance with the fair credit reporting a...,accord fair credit report act list account vio...
263299,Debt collection,Communication tactics,GA,30342.0,2161130,I am receiving multiple daily calls on my land...,receiv multipl daili call land line portfolio ...
219871,Credit card,Arbitration,KS,67205.0,2201117,I requested a mediator from American Express X...,request mediat american express month ago rega...
1018210,Mortgage,Trouble during payment process,NJ,8406.0,4136129,I purchased my home in XXXX of XXXX. NewRez to...,purchas home newrez took loan origin mortgag c...


Wall time: 2.07 s


In [21]:
small_cfpb_df

""
